Re-run the optimization of the MERA (for small bond dimensions - only 2 -, even if we don't get the correct energy)\
contract the correct indices and then save them into an appropriate file

In [1]:
import quimb as qu
import quimb.tensor as qtn
import matplotlib.pyplot as plt
import numpy as np
import cotengra as ctg

from mera_hubbard import FH_Hamiltonian_NN_half_filling, solve_ground_state

/Users/bai/local/src/MERA-Hubbard/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
num_sites = 2
t1 = 1.
t2 = 0.5 # trying without the NN terms first, as additional check
U = 2.

# FH HAMILTONIAN
ham, sparse_ham = FH_Hamiltonian_NN_half_filling(num_sites, t1, t2, U, pbc=0) # pbc=0: open BC, pbc=1: cyclic BC
ham.show()
en, _, _ = solve_ground_state(num_sites, ham, sparse_ham)

def norm_fn(mera):
    # there are a few methods to do the projection
    # exp works well for optimization
    return mera.isometrize(method='exp')

def expectation_sites(mera, local_terms, optimize='auto-qt'):
    """Compute the energy given the mera and the local terms of the hamiltonian
    """
    # if we pass directly the hamiltonian the optimizer complains...
    energy = 0.

    for key in local_terms:
        sites = []

        term = local_terms[key]
        
        for item in key:
            if item[0] == '↑':
                sites.append(item[1]-1)
            elif item[0] == '↓':
                sites.append(item[1]-1 + int(ham.nsites/2))

        # what gate should we apply?? (instead of 'term')
        mera_op = mera.gate(term, sites, propagate_tags=False)
        
        mera_ex = mera_op & mera.H # apply the h.c. of the mera to calculate the expectation value

        energy += mera_ex.contract(all, optimize=optimize) # calculate the expect. value
    
    return energy


Reference ED at half-filling:


/Users/bai/local/src/MERA-Hubbard/.venv/lib/python3.12/site-packages/quimb/experimental/operatorbuilder/operatorbuilder.py:1431: NumbaTypeSafetyWarning: unsafe cast from int64 to undefined. Precision may be lost.
  ci = bitmap[bi]


energy =  -1.2360679774997894
number of particles =  2.0


New Hamiltonian at half-filling:
chemical potential at half-filling =  -0.882
SparseOperatorBuilder(nsites=4, nterms=10, locality=2))
+ - . .  -1.0
- + . .  -1.0
. . + -  -1.0
. . - +  -1.0
n . n .  +2.0
sn. . .  -0.8820000000001231
. . sn.  -0.8820000000001231
. n . n  +2.0
. sn. .  -0.8820000000001231
. . . sn -0.8820000000001231
energy =  -1.2360679774997898
number of particles =  2.0


In [3]:
max_bond = 2
n = num_sites * 2
print(f'\n*************** OPTIMIZATION MERA - MAX_BOND = {max_bond} ***************\n')
# Initialization MERA
mera = qtn.MERA.rand(n, max_bond=max_bond) #, dangle=True) # the dangle option determines if the last isometry has an external edge not connected to anything else.. it affects the dimension of the tensor!!!
mera.isometrize_()

# build local terms to then compute the energy term by term (needed for the optimization of the MERA)
local_terms = ham.build_local_terms()

# To find a high quality contraction path for each term 
opt = ctg.ReusableHyperOptimizer(
    progbar=True,
    reconf_opts={},
    max_repeats=16,
    # directory=  # set this for persistent cache
)

expectation_sites(mera, local_terms, optimize=opt)
expectation_sites(mera, local_terms, optimize=opt)

# REAL OPTIMIZATION
# set-up the MERA optimizer object:
tnopt = qtn.TNOptimizer(
    mera,
    loss_fn=expectation_sites, 
    norm_fn=norm_fn,
    loss_constants={'local_terms': local_terms},
    loss_kwargs={'optimize': opt},
    autodiff_backend='torch', device='cpu', jit_fn=True, 
)

# the first step involves compiling the computation, which might take some time and print some (ignorable) warnings:
tnopt.optimize(1)

tnopt.optimizer = 'l-bfgs-b'  # the default
mera_opt_hubbard = tnopt.optimize(999)

tnopt.optimizer = 'adam'  # useful for final iterations
mera_opt_hubbard = tnopt.optimize(1000)    

en_mera = expectation_sites(mera_opt_hubbard, local_terms, optimize=opt)

/Users/bai/local/src/MERA-Hubbard/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:54: UserWarning: Couldn't find `optuna`, `baytune (btb)`, `chocolate`, `nevergrad` or `skopt` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(



*************** OPTIMIZATION MERA - MAX_BOND = 2 ***************



  0%|          | 0/16 [00:00<?, ?it/s]/Users/bai/local/src/MERA-Hubbard/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/Users/bai/local/src/MERA-Hubbard/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/Users/bai/local/src/MERA-Hubbard/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
F=2.66 C=3.93 S=4.00 P=7.29:   6%|▋         | 1/16 [00:00<00:03,  3.77it/s]/Users/bai/local/src/MERA-Hubbard/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skippin

ModuleNotFoundError: No module named 'torch'

In [ ]:
# select the tensors layer by layer
new_tensors = []
tensors_dict = {"tensors": [], "qubits": [], "tags": []}
jump = 1
ind = 0
start_ind = 0
for n_layer in np.arange(0,np.log2(mera_opt_hubbard.nsites)):
    # print(f'************* _LAYER{int(n_layer)} *************')
    mera_layer = mera_opt_hubbard.select([f'_LAYER{int(n_layer)}'], 'all')
    node_shape = dict([('_UNI','o'), ('_ISO','v')])
    # mera_layer.draw(color=['_UNI', '_ISO'], show_inds='all', show_tags=False, node_shape=node_shape)
    
    # select only the unitaries
    mera_l_uni = mera_layer.select(['_UNI'], 'all')

    for tn in mera_l_uni.tensors:
        # tn = tn.data
        # if '_UNI' in tn.tags: 
        ind_k1 = tn.inds[0]
        ind_k2 = tn.inds[1]
        ind_ex1 = tn.inds[2]
        ind_ex2 = tn.inds[3]
        new_tn = tn.fuse({f'ind_int':(f'{ind_k1}', f'{ind_k2}')})
        new_tn = new_tn.fuse({f'ind_ext':(f'{ind_ex1}', f'{ind_ex2}')})
        # print(new_tn)
        new_tensors.append(new_tn.data.tolist())
        if ind+jump >= n:
            tensors_dict["tensors"].append(new_tn.data.tolist())
            tensors_dict["qubits"].append([ind, ind+jump-n])
            tensors_dict["tags"].append("UNI")
        else:
            tensors_dict["tensors"].append(new_tn.data.tolist())
            tensors_dict["qubits"].append([ind, ind+jump])
            tensors_dict["tags"].append("UNI")
        ind += (jump*2)
        if ind >= n:
            ind = ind - n
        # print('UNI appended')

    # select only the isometries
    mera_l_iso = mera_layer.select(['_ISO'], 'all')
    
    ind = start_ind + jump
    for tn in mera_l_iso.tensors:
        # tn = tn.data
        # if '_ISO' in tn.tags: 
        if len(tn.inds) >= 3:
            ind_in1 = tn.inds[0]
            ind_in2 = tn.inds[1]
            ind_ex = tn.inds[2]
            new_tn = tn.fuse({f'ind_int':(f'{ind_in1}', f'{ind_in2}')})
            # print(new_tn.data)
            v1 = new_tn.data[:,0]
            v2 = new_tn.data[:,1]
            rnd1 = np.random.randn(4)  # take a random vector
            rnd2 = np.random.randn(4)  # take a random vector

            u0 = v1 # cannot change
            u1 = v2 # cannot change
            u2 = rnd1 - np.dot(u0,rnd1)*u0 - np.dot(u1,rnd1)*u1
            u3 = rnd2 - np.dot(u0,rnd2)*u0 - np.dot(u1,rnd2)*u1 - np.dot(u2,rnd2)*u2/np.dot(u2,u2)
            u2 /= np.linalg.norm(u2)  # normalize it
            u3 /= np.linalg.norm(u3)  # normalize it

            new_tn_completed = np.zeros((4,4))
            new_tn_completed[:,0:2] = new_tn.data
            new_tn_completed[:,2] = u2
            new_tn_completed[:,3] = u3
            # print(new_tn_completed)

        else: # last tensor, it has one index less
            ind_in1 = tn.inds[0]
            ind_in2 = tn.inds[1]
            new_tn = tn.fuse({f'ind_int':(f'{ind_in1}', f'{ind_in2}')})
            # print(new_tn.data)
            v1 = new_tn.data
            rnd1 = np.random.randn(4)  # take a random vector
            rnd2 = np.random.randn(4)  # take a random vector
            rnd3 = np.random.randn(4)  # take a random vector

            u0 = v1 # cannot change
            u1 = rnd1 - np.dot(u0,rnd1)*u0
            u2 = rnd2 - np.dot(u0,rnd2)*u0/np.dot(u0,u0) - np.dot(u1,rnd2)*u1/np.dot(u1,u1)
            u3 = rnd3 - np.dot(u0,rnd3)*u0/np.dot(u0,u0) - np.dot(u1,rnd3)*u1/np.dot(u1,u1) - np.dot(u2,rnd3)*u2/np.dot(u2,u2)
            u1 /= np.linalg.norm(u1)  # normalize it
            u2 /= np.linalg.norm(u2)  # normalize it
            u3 /= np.linalg.norm(u3)  # normalize it

            new_tn_completed = np.zeros((4,4))
            new_tn_completed[:,0] = new_tn.data
            new_tn_completed[:,1] = u1
            new_tn_completed[:,2] = u2
            new_tn_completed[:,3] = u3
                        
            # new_tn_completed = tn.data
        new_tensors.append(new_tn_completed.tolist())
        if ind+jump >= n:
            tensors_dict["tensors"].append(new_tn_completed.data.tolist())
            tensors_dict["qubits"].append([ind, ind+jump-n])
            tensors_dict["tags"].append("ISO")
        else:
            tensors_dict["tensors"].append(new_tn_completed.data.tolist())
            tensors_dict["qubits"].append([ind, ind+jump])
            tensors_dict["tags"].append("ISO")
        ind += (jump*2)
        if ind >= n:
            ind = ind - n
        # print('ISO appended')
    
    start_ind += int(2**n_layer)
    jump += int(2**n_layer)


In [6]:
# tensors_dict

In [7]:
# new_tensors = []
# previous_inds = []
# for i, tn in enumerate(mera_opt_hubbard.tensors):
#     tags = []
#     for t in tn.tags:
#         tags.append(t)
#     # print(tags)
    
#     if '_UNI' in tags: # and '_LAYER0' in tags:
#         ind_k1 = tn.inds[0]
#         ind_k2 = tn.inds[1]
#         ind_ex1 = tn.inds[2]
#         ind_ex2 = tn.inds[3]
#         new_tn = tn.fuse({f'ind_k_{i}':(f'{ind_k1}', f'{ind_k2}')})
#         new_tn = new_tn.fuse({f'ind_ex_{i}':(f'{ind_ex1}', f'{ind_ex2}')})
#         previous_inds.append([ind_ex1, ind_ex2])
#         # print(new_tn)
#         new_tensors.append(new_tn.data.tolist())

#     if '_ISO' in tags: # and '_LAYER0' in tags:

#         if len(tn.inds) >= 3:
#             ind_in1 = tn.inds[0]
#             ind_in2 = tn.inds[1]
#             ind_ex = tn.inds[2]
#             new_tn = tn.fuse({f'ind_in_{i}':(f'{ind_in1}', f'{ind_in2}')})
#             previous_inds.append([])
#             # print(new_tn.data)
#             v1 = new_tn.data[:,0]
#             v2 = new_tn.data[:,1]
#             rnd1 = np.random.randn(4)  # take a random vector
#             rnd2 = np.random.randn(4)  # take a random vector

#             u0 = v1 # cannot change
#             u1 = v2 # cannot change
#             u2 = rnd1 - np.dot(u0,rnd1)*u0 - np.dot(u1,rnd1)*u1
#             u3 = rnd2 - np.dot(u0,rnd2)*u0 - np.dot(u1,rnd2)*u1 - np.dot(u2,rnd2)*u2/np.dot(u2,u2)
#             u2 /= np.linalg.norm(u2)  # normalize it
#             u3 /= np.linalg.norm(u3)  # normalize it

#             new_tn_completed = np.zeros((4,4))
#             new_tn_completed[:,0:2] = new_tn.data
#             new_tn_completed[:,2] = u2
#             new_tn_completed[:,3] = u3
#             # print(new_tn_completed)

#         else: # last tensor, it has one index less
#             ind_in1 = tn.inds[0]
#             ind_in2 = tn.inds[1]
#             new_tn = tn.fuse({f'ind_in_{i}':(f'{ind_in1}', f'{ind_in2}')})
#             previous_inds.append([])
#             # print(new_tn.data)
#             v1 = new_tn.data
#             rnd1 = np.random.randn(4)  # take a random vector
#             rnd2 = np.random.randn(4)  # take a random vector
#             rnd3 = np.random.randn(4)  # take a random vector

#             u0 = v1 # cannot change
#             u1 = rnd1 - np.dot(u0,rnd1)*u0
#             u2 = rnd2 - np.dot(u0,rnd2)*u0/np.dot(u0,u0) - np.dot(u1,rnd2)*u1/np.dot(u1,u1)
#             u3 = rnd3 - np.dot(u0,rnd3)*u0/np.dot(u0,u0) - np.dot(u1,rnd3)*u1/np.dot(u1,u1) - np.dot(u2,rnd3)*u2/np.dot(u2,u2)
#             u1 /= np.linalg.norm(u1)  # normalize it
#             u2 /= np.linalg.norm(u2)  # normalize it
#             u3 /= np.linalg.norm(u3)  # normalize it

#             new_tn_completed = np.zeros((4,4))
#             new_tn_completed[:,0] = new_tn.data
#             new_tn_completed[:,1] = u1
#             new_tn_completed[:,2] = u2
#             new_tn_completed[:,3] = u3
                        
#             # new_tn_completed = tn.data
#         new_tensors.append(new_tn_completed.tolist())

#         # checks 
#         # print(np.round(np.dot(new_tn_completed.transpose(),new_tn_completed),3))
#         # print(np.round(np.dot(new_tn_completed,new_tn_completed.transpose())))

#         # print('det = ', np.linalg.det(new_tn_completed))
#         # print('norm row = ', np.linalg.norm(new_tn_completed[2]))

In [8]:
# save the dictionary as .json file, easier to upload in qiskit..
import json

# mera_tensors = dict()
# for ind, tn in enumerate(new_tensors):
#     # print(tn.left_inds) # indices of the edges that 'enter' the node
#     # print(tn.get_params())
#     mera_tensors[ind] = tn

# Writing to sample.json
with open(f"not-optMERA_1x{num_sites}_U={U}_t2={t2}.json", "w") as outfile:
    # Serializing json
    json_object = json.dumps(tensors_dict)
    outfile.write(json_object)

In [9]:
# # Opening JSON file
# with open(f"not-optMERA_1x{num_sites}_U={U}_t2={t2}.json", 'r') as openfile:
#     # Reading from json file
#     mera_json = json.load(openfile)
